In [5]:
!rm -rf Eunomia
!git clone https://github.com/ouaguenouni/Eunomia
!pip install -r Eunomia/requirements.txt

Cloning into 'Eunomia'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 97 (delta 47), reused 72 (delta 22), pack-reused 0
Receiving objects: 100% (97/97), 57.05 KiB | 2.19 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from Eunomia.preferences import *
from Eunomia.additive_functions import *
from Eunomia.alternatives import *
from Eunomia.sampling import *
from Eunomia.mcmc import *
from Eunomia.degree import *
from Eunomia.experiments import *

pyro.set_rng_seed(1)

In [ ]:
import time

TEST_NAME = "SX1"

REPLACE = False

n_vals = [5,6,7,8,9,10]
sigma_weights = 10
sigma_w_vals = [1, 1e-1, 1e-2, 1e1]
sigma_p_vals = [1, 1e-1, 1e-2, 1e1]
n_samples = 1500


for n in n_vals:
    for m in list(2**i for i in range(2, n)):
        for k in range(1, n-1):
                for sigma_w in sigma_w_vals:
                    for sigma_p in sigma_p_vals:
                        
                        run_d = {
                            "n": n,
                            "k": k,
                            "m": m,
                            "sigma_w": sigma_w,
                            "sigma_p": sigma_p,
                            "n_samples": n_samples
                        }

                        found = find_experiment_file(run_d, TEST_NAME)

                        if found:
                            date = found
                            print(f"Found file: {file_name}")

                            if REPLACE:
                                print('Replacing it...')
                            else:
                                continue
                        else:
                            file_name = compute_experiment_file_name(run_d, TEST_NAME)
                            print("Registering the results in: ", file_name)

                        theta = generate_additive_theta(n, k)
                        weights = generate_normal_weights(theta, sigma_weights)
                        alternatives = generate_random_alternatives_matrix(m, n)
                        ranks = compute_ws_ranks(alternatives, theta, weights)
                        t_sv = compute_semivalues(n, theta, weights, lambda x:1)
                        preferences = PreferenceModel(alternatives, ranks)
                        data = preferences.generate_preference_matrix(theta)
                        data = torch.tensor(data).float()
                        t = time.time()
                        model = posterior_sampling_model(data, sigma_w=sigma_w, sigma_p=sigma_p)
                        diag, sampled_weights, sigmas = sample_model(model, data, "w", "sigma", warmup_steps=500, num_samples=n_samples, return_diag=True)
                        t = time.time() - t
                        accs_d = get_acc_distribution(data, sampled_weights, sigmas)

                        predicted_rankings = [np.argsort(compute_semivalues(n, theta, weights, lambda x:1))[::-1] for weights in sampled_weights]
                        kt_d = get_kt_distribution(predicted_rankings, np.argsort(t_sv))

                        file_path = record_experiment_results(run_d, TEST_NAME)
                        run_d["time"] = t
                        run_d["weights"] = weights.tolist()
                        run_d["predicted_rankings"] = [i.tolist() for i in predicted_rankings]
                        run_d["accuracy_distribution"] = [i.tolist() for i in accs_d]
                        run_d["kt_d"] = [i.tolist() for i in kt_d]
                        run_d["acceptance_rate"] = diag["acceptance rate"]["chain 0"]
                        run_d["w_eff"] = [float(i) for i in diag["w"]["n_eff"].numpy()]
                        run_d["w_rhat"] = [float(i) for i in diag["w"]["r_hat"].numpy()]

                        print("diag: ", diag)
                        with open(file_path, 'w') as file:
                            yaml.dump(run_d, file, default_flow_style=False)



Registering the results in:  SX1/2023-11-18/n-5_k-1_m-4_sigma_w-1_sigma_p-1_n_samples-1500.yaml


Sample: 100%|████████████████████████████| 2000/2000 [01:36, 20.62it/s, step size=1.70e-01, acc. prob=0.907]


diag:  {'sigma': OrderedDict([('n_eff', tensor(821.4919)), ('r_hat', tensor(0.9998))]), 'w': OrderedDict([('n_eff', tensor([810.8021, 823.4952, 878.3043, 877.1406, 565.9174])), ('r_hat', tensor([1.0001, 0.9994, 1.0039, 1.0000, 0.9997]))]), 'divergences': {'chain 0': [175, 501, 558, 662, 665, 666, 677, 680, 685, 780, 789, 1213, 1478]}, 'acceptance rate': {'chain 0': 0.9933333333333333}}
Registering the results in:  SX1/2023-11-18/n-5_k-1_m-4_sigma_w-1_sigma_p-0.1_n_samples-1500.yaml


Sample: 100%|████████████████████████████| 2000/2000 [00:48, 41.51it/s, step size=3.72e-01, acc. prob=0.895]


diag:  {'sigma': OrderedDict([('n_eff', tensor(910.2490)), ('r_hat', tensor(1.0024))]), 'w': OrderedDict([('n_eff', tensor([ 962.2154,  914.4397, 1273.5034, 1261.7496, 1054.6635])), ('r_hat', tensor([0.9994, 1.0040, 0.9994, 1.0019, 0.9996]))]), 'divergences': {'chain 0': [6, 8, 57, 701, 801, 1155, 1157, 1160, 1162, 1172, 1389, 1459]}, 'acceptance rate': {'chain 0': 0.978}}
Registering the results in:  SX1/2023-11-18/n-5_k-1_m-4_sigma_w-1_sigma_p-0.01_n_samples-1500.yaml


...

Sample: 100%|████████████████████████████| 2000/2000 [01:53, 17.55it/s, step size=9.75e-02, acc. prob=0.716]


diag:  {'sigma': OrderedDict([('n_eff', tensor(366.5648)), ('r_hat', tensor(1.0011))]), 'w': OrderedDict([('n_eff', tensor([ 472.8763,  805.8082,  634.6381,  955.7746,  811.9014,  590.6646,
         908.6186,  652.3631,  683.5454,  475.6772,  497.6592,  568.5312,
         455.2356,  691.0026,  691.8518,  933.8276,  810.9117,  561.6003,
         954.9142,  741.9510,  497.2172,  743.8358,  821.7717,  537.4445,
        1060.5544,  964.8417, 1017.9377, 1069.9541,  499.9042,  760.1219,
         717.4867,  770.4538, 1174.5807,  955.9658, 1180.7058,  557.6371,
        1005.1973,  757.8813,  537.7061,  584.7090,  765.8378,  526.7404,
         982.2812,  751.5035,  975.1987,  893.3240,  573.5519,  931.2097,
         549.8864,  789.6983,  537.6848,  752.1894,  580.9171,  693.1594,
         844.4647, 1142.6216])), ('r_hat', tensor([0.9998, 0.9995, 1.0000, 1.0000, 0.9994, 0.9994, 0.9994, 1.0000, 0.9997,
        1.0012, 1.0007, 0.9997, 1.0002, 0.9994, 0.9993, 0.9996, 0.9996, 0.9996,
        1.0014,

Sample: 100%|████████████████████████████| 2000/2000 [02:00, 16.55it/s, step size=7.77e-02, acc. prob=0.814]


diag:  {'sigma': OrderedDict([('n_eff', tensor(107.0777)), ('r_hat', tensor(1.0188))]), 'w': OrderedDict([('n_eff', tensor([ 279.2449,  264.1817, 1337.1633, 1102.7197, 1014.4617, 1009.0999,
        1297.6251,  494.6512,  708.3409, 1097.0702,  685.8394,  524.8637,
         568.4676,  312.7893,  687.0889,  422.6514, 1168.0974,  927.3365,
         531.3776,  866.8712,  611.7709,  537.0356,  909.8967, 1409.9285,
         651.5184,  537.0331, 1014.6049,  709.6889,  798.9111,  655.2649,
         580.8084, 1026.6396,  485.0591,  400.6620,  782.2908,  657.7437,
         198.1607, 1950.8433,  508.7551,  767.4060,  192.7249, 1044.3903,
        1420.3154, 1411.1479,  278.3634, 1566.4392,  936.4387, 1041.5145,
         604.3523, 1692.1046,  697.6591,  486.2647,  744.3118,  264.5369,
         466.8637,  255.3031])), ('r_hat', tensor([1.0019, 1.0044, 0.9999, 0.9996, 1.0001, 1.0002, 0.9994, 0.9993, 1.0011,
        0.9997, 0.9994, 1.0026, 1.0005, 0.9997, 0.9994, 1.0042, 0.9995, 1.0007,
        1.0051,

Sample:  51%|██████████████▏             | 1013/2000 [05:07,  3.97it/s, step size=1.65e-02, acc. prob=0.777]

In [ ]:
preferences = PreferenceModel(alternatives, ranks)
data = preferences.generate_preference_matrix(theta)